# Nextflow

[Nextflow](https://www.nextflow.io/) is a workflow management system used for executing scientific workflows across platforms scalably, portably, and reproducibly.

Here, we'll run a demo of the microscopy pipeline [mcmicro](https://github.com/labsyspharm/mcmicro) to correct uneven illumination. [Reference](https://mcmicro.org/)


```{note}

This notebook serves as a demo for Python scripting that you could run before and after Nextflow runs.

Typically, you'd run the workflows from the command line or Nextflow Tower and register data from a Python script and not necessarily a notebook.
```

## Setup

Let's create a test instance:

In [ ]:
!lamin load nextflow-mcmicro

In [ ]:
import lamindb as ln
from subprocess import getoutput

## Track the Nextflow run

Track the Nextflow workflow & run:

In [ ]:
transform = ln.Transform(
    name="mcmicro",
    version="1.0.0",
    type="pipeline",
    reference="https://github.com/labsyspharm/mcmicro",
)
ln.track(transform)
# grab the run of the global run context
run = ln.dev.run_context.run

If we now stage input files, they'll be tracked as inputs for the global run:

In [ ]:
# we fetch example input files that are already stored in the instance
mcmicro_input = ln.File.filter(key__startswith="exemplar-001")
input_paths = [input_fastq.stage() for input_fastq in mcmicro_input]

Run the nextflow pipeline:

In [ ]:
!nextflow run https://github.com/labsyspharm/mcmicro --in exemplar-001 --start-at illumination --stop-at registration -name "lamin_{run.uid}"

Here, we passed the LaminDB run id to nextflow so that we can query it from within nextflow.

## Register outputs

In [ ]:
mcmicro_registered_File = ln.File("exemplar-001/registration/exemplar-001.ome.tif")
mcmicro_registered_File.save()

## Track Nextflow ID

Let us look at the nextflow logs:

In [ ]:
!nextflow log

Let us add the information about the session ID to our `run` record:

In [ ]:
nextflow_id = getoutput(f"nextflow log | awk '/{run.id}/{{print $8}}'")
run.reference = nextflow_id
run.reference_type = "nextflow_id"
run.save()

## Data lineage

View data lineage:

In [ ]:
mcmicro_registered_File.view_flow()

View the database content:

In [ ]:
ln.view()

Clean up the test instance:

In [ ]:
!lamin delete --force nextflow-mcmicro

If you are interested in registering bulk RNA-seq data with Lamin, you can have a look at our [nf-core/rnaseq example](https://github.com/laminlabs/nextflow-lamin-usecases/blob/main/nextflow_bulk_rna.ipynb).